In [6]:
import pandas as pd
import os
import glob

# Path to the data folder
data_folder = "C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data"

# Find all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

if not csv_files:
    print("No CSV files found in the folder.")
    exit()

dfs = []

for file in csv_files:
    if not os.path.isfile(file):
        print(f"⏩ Skipping non-file: {file}")
        continue

    try:
        # Always use latin-1 encoding
        df = pd.read_csv(file, encoding='latin-1')
        print(f"✅ Successfully read {file} using latin-1 encoding.")
    except Exception as e:
        print(f"❌ Error reading {file}: {e}")
        continue

    # Clean column names
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("[^0-9a-zA-Z_]", "", regex=True)
    )
    
    # Optional: add the source file column
    df["source_file"] = os.path.basename(file)
    
    dfs.append(df)

# Merge all dataframes
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv("merged_cleaned_data.csv", index=False)
    print("✅ Merged and cleaned data saved to 'merged_cleaned_data.csv'")
else:
    print("❌ No dataframes were created. Please check the input CSV files.")
 

✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_0700_NIC_FINAL_STATE_NCT_OF_DELHI-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_1600_NIC_FINAL_STATE_TRIPURA-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_2000_NIC_FINAL_STATE_JHARKHAND-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_2400_NIC_FINAL_STATE_GUJARAT-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_2700_NIC_FINAL_STATE_MAHARASHTRA-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/data\DDW_B18sc_2900_NIC_FINAL_STATE_KARNATAKA-2011.csv using latin-1 encoding.
✅ Successfully read C:/Users/Mansi/.vscode/industrial-hr-geo-visualisation/

In [7]:
import pandas as pd

# Load merged CSV
df = pd.read_csv("merged_cleaned_data.csv")

# --- Fix `indiastates` and remove backticks if any ---
df["indiastates"] = df["indiastates"].str.replace("STATE - ", "", regex=False).str.strip()
df["indiastates"] = df["indiastates"].str.replace("`", "", regex=False).str.title()

# --- Create Year Column ---
df["year"] = 2011

# --- Extract NIC Codes as Strings (Remove backticks and strip) ---
for col in ["division", "group", "class"]:
    df[col] = df[col].astype(str).str.replace("`", "", regex=False).str.zfill(2)

# --- Rename and Clean Worker Columns for Easier Access ---
df = df.rename(columns={
    "main_workers__total__males": "main_males",
    "main_workers__total__females": "main_females",
    "marginal_workers__total__males": "marginal_males",
    "marginal_workers__total__females": "marginal_females",
})

# --- Compute Total Main & Marginal & Overall Workers ---
df["total_main_workers"] = df["main_males"] + df["main_females"]
df["total_marginal_workers"] = df["marginal_males"] + df["marginal_females"]
df["total_workers"] = df["total_main_workers"] + df["total_marginal_workers"]

# --- Clean Industry Type ---
df["industry_type"] = df["nic_name"].str.title().str.strip()

# --- Create Industry Group ---
def classify_industry(name):
    if pd.isna(name): return "Unknown"
    name = name.lower()
    if "crop" in name or "animal" in name or "farming" in name or "agriculture" in name:
        return "Agriculture"
    elif "manufacturing" in name:
        return "Manufacturing"
    elif "mining" in name:
        return "Mining"
    elif "construction" in name:
        return "Construction"
    elif "trade" in name or "retail" in name or "wholesale" in name:
        return "Trade & Commerce"
    elif "transport" in name or "logistics" in name:
        return "Transport"
    elif "education" in name:
        return "Education"
    elif "health" in name:
        return "Health"
    elif "administration" in name or "government" in name:
        return "Public Administration"
    elif "support" in name or "service" in name:
        return "Support Services"
    else:
        return "Other"

df["industry_group"] = df["industry_type"].apply(classify_industry)

# --- Save Engineered File ---
df.to_csv("merged_feature_engineered.csv", index=False)
print("✅ Feature engineering complete. File saved as 'merged_feature_engineered.csv'")



✅ Feature engineering complete. File saved as 'merged_feature_engineered.csv'


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import streamlit as st
import os

# Load data
df = pd.read_csv("merged_feature_engineered.csv")

# Create folder to save outputs
output_dir = "eda_charts"
os.makedirs(output_dir, exist_ok=True)

# Clean and set up
sns.set(style="whitegrid")

# -----------------------
# 1️⃣ Workers by Industry
# -----------------------
industry_counts = df.groupby("industry_group")["total_workers"].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
industry_df = industry_counts.reset_index().rename(columns={"total_workers": "Total Workers", "industry_group": "Industry"})
sns.barplot(data=industry_df, x="Total Workers", y="Industry", hue="Industry", palette="viridis", legend=False)
plt.title("Total Workers by Industry Group")
plt.xlabel("Total Workers")
plt.tight_layout()
plt.savefig(f"{output_dir}/industry_worker_distribution.png")
plt.close()

# ------------------------
# 2️⃣ Top 15 States by Workers
# ------------------------
state_counts = df.groupby("indiastates")["total_workers"].sum().sort_values(ascending=False).head(15)
plt.figure(figsize=(12, 6))
state_df = state_counts.reset_index().rename(columns={"total_workers": "Total Workers", "indiastates": "State"})
sns.barplot(data=state_df, x="Total Workers", y="State", hue="State", palette="cubehelix", legend=False)
plt.title("Top 15 States by Total Workers")
plt.xlabel("Total Workers")
plt.tight_layout()
plt.savefig(f"{output_dir}/state_worker_distribution.png")
plt.close()

# -------------------------------
# 3️⃣ Gender-wise Worker Patterns
# -------------------------------
gender_df = df[["indiastates", "main_males", "main_females", "marginal_males", "marginal_females"]]
gender_summary = gender_df.groupby("indiastates").sum().sort_values("main_males", ascending=False).head(15)

# Main Workers
gender_summary[["main_males", "main_females"]].plot(kind="bar", stacked=True, colormap="coolwarm", figsize=(14, 6))
plt.title("Main Workers by Gender (Top 15 States)")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"{output_dir}/main_workers_gender.png")
plt.close()

# Marginal Workers
gender_summary[["marginal_males", "marginal_females"]].plot(kind="bar", stacked=True, colormap="Set2", figsize=(14, 6))
plt.title("Marginal Workers by Gender (Top 15 States)")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"{output_dir}/marginal_workers_gender.png")
plt.close()

# -------------------------------
# 4️⃣ Dominant Industry per State
# -------------------------------
dominant_industries = df.groupby(["indiastates", "industry_group"])["total_workers"].sum().reset_index()
dominant_top = dominant_industries.sort_values(["indiastates", "total_workers"], ascending=[True, False])
dominant_top = dominant_top.groupby("indiastates").head(1)

plt.figure(figsize=(12, 6))
sns.barplot(data=dominant_top, y="indiastates", x="total_workers", hue="industry_group", dodge=False)
plt.title("Dominant Industry by State (Highest Total Workers)")
plt.tight_layout()
plt.savefig(f"{output_dir}/dominant_industry_by_state.png")
plt.close()

# -------------------------------------
# 5️⃣ Major vs Marginal Worker Analysis
# -------------------------------------
worker_type_df = df.groupby("indiastates")[["total_main_workers", "total_marginal_workers"]].sum()
worker_type_df = worker_type_df.sort_values("total_main_workers", ascending=False).head(15)
worker_type_df.plot(kind="bar", stacked=True, colormap="Spectral", figsize=(14, 6))
plt.title("Main vs Marginal Workers (Top 15 States)")
plt.ylabel("Total Workers")
plt.tight_layout()
plt.savefig(f"{output_dir}/main_vs_marginal_workers.png")
plt.close()

# --------------------------------------------------
# 6️⃣ Worker Distribution Histogram (Optional Insight)
# --------------------------------------------------
plt.figure(figsize=(10, 5))
sns.histplot(df["total_workers"], bins=50, kde=True, color="teal")
plt.title("Distribution of Total Workers Across All Records")
plt.xlabel("Total Workers")
plt.tight_layout()
plt.savefig(f"{output_dir}/worker_distribution_histogram.png")
plt.close()

# ---------------------------------------------------
# 7️⃣ Interactive Alternative (Optional - Clean Layout)
# ---------------------------------------------------
top_industry_states = df.groupby(["indiastates", "industry_group"])["total_workers"].sum().reset_index()
top_industry_states = top_industry_states.sort_values("total_workers", ascending=False).head(30)

fig = px.bar(top_industry_states, x="total_workers", y="indiastates", color="industry_group",
             orientation='h', title="Top 30 Industry Groups by State")
fig.write_html(f"{output_dir}/interactive_barplot.html")

with open(f"{output_dir}/interactive_barplot.html", 'r', encoding='utf-8') as f:
    html_data = f.read()
    st.components.v1.html(html_data, height=600, scrolling=True)



2025-07-12 21:17:07.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 21:17:07.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
